In [1]:
# MLM 방식을 이용한 Further pre-traning 방식 구현 예제
# 참고 소스 : https://towardsdatascience.com/masked-language-modelling-with-bert-7d49793e5d2c 참조 바람
import torch
import os

from tqdm.notebook import tqdm
from transformers import BertTokenizer, BertConfig, BertForMaskedLM, BertTokenizerFast
from transformers import AdamW, get_linear_schedule_with_warmup
from os import sys
#sys.path.append('..')
from myutils import GPU_info, seed_everything, mlogging

logfilepath:bwdataset_2022-04-15.log
logfilepath:qnadataset_2022-04-15.log


In [2]:
#======================================================================================================
# *WandB 툴을 이용하여, HyperPrameter sweeps 하는 예제임(http://wandb.ai 에 회원가입 해야 함)
#  
# == HyperPrameter sweeps 과정 ==
#
# 1. 로그인
# import wandb
# wandb.login()  # 로그인 => wandb.ai 사이트에 로그인 후, API Key 입력해야 함
#
# 2. sweep_config 설정
# sweep_config ={ 
#       'method': 'random', #grid, random
#       'metric':{
#           'name': 'val_accuracy', #loss
#           'goal': 'maximize'      # loss일때는 minimize로 설정해야함
#       },
#      'parameters':{
#           'learning_rate':{
#               'distribution': 'uniform',
#               'min' : 0,
#               'max': 7e-5
#           },
#           'batch_size':{
#               'values' : [8, 16, 32]
#           },
#           'epochs':{
#               'values' : [2, 3, 4]
#           }
#       }
#}
#
#sweep_id = wandb.sweep(sweep_config, project="bert-wb-test")
#
# 3. wandb 초기화 
# - wandb.init(config=config)
#
# 4. wandb 로그설정
# - wandb.log({"val_accuracy": total_test_correct / total_test_len})
#
# 5. wand 실행
# - wandb.agent(sweep_id, train, count=3)
#
# 6. wand 종료
# -wandb.finish()
#======================================================================================================
# 1. 로그인
#!pip install wandb -qqq
import wandb
wandb.login()  # 로그인 => wandb.ai 사이트에 로그인 후, API Key 입력해야 함

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kobongsoo (use `wandb login --relogin` to force relogin)


True

In [3]:
# 2. sweep_config 설정

sweep_config ={ 
      'method': 'random', #grid, random
      'metric':{
          'name': 'val_accuracy', #loss
          'goal': 'maximize'      # loss일때는 minimize로 설정해야함
      },
     'parameters':{
          'learning_rate':{
              'distribution': 'uniform',
              'min' : 0,
              'max': 7e-5
          },
          'batch_size':{
              'values' : [8, 16, 32]
          },
          'epochs':{
              'values' : [3, 4, 5]
          }
      }
}

sweep_id = wandb.sweep(sweep_config, project="bert-wb-mlm-test")

Create sweep with ID: 4fsqluk6
Sweep URL: https://wandb.ai/kobongsoo/bert-wb-mlm-test/sweeps/4fsqluk6


In [4]:
# 훈련시킬 말뭉치(사전 만들때 동일한 말뭉치 이용)
input_corpus = "../korpora/kowiki_20190620/wiki_20190620_small.txt"

# test 말뭉치 
eval_corpus = "../korpora/kowiki_20190620/wiki_eval_test.txt"

# 기존 사전훈련된 모델
model_path = "../model/bert/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0327/"

# 기존 사전 + 추가된 사전 파일
#vocab_path="tokenizer/wiki_20190620_false_0311_speical/bmc_add_wiki_20190620_false_0311.txt"
vocab_path="../model/bert/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0327/"

# 출력
OUTPATH = '../model/bert/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0414/'

#batch_size = 32
token_max_len = 128

device = GPU_info()
print(device)

#seed 설정
seed_everything(222)

#logging 설정
logger =  mlogging(loggername="wb-bert-fpt-mlm", logfilename="../log/wb-bert-fpt-mlm")

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../log/wb-bert-fpt-mlm_2022-04-15.log


In [5]:
# tokeinzier 생성
# tokenizer 생성
# => BertTokenizer, BertTokenizerFast 둘중 사용하면됨

#tokenizer = BertTokenizer(vocab_file=vocab_path, max_len=token_max_len, do_lower_case=False)
tokenizer = BertTokenizer.from_pretrained(vocab_path, max_len=token_max_len, do_lower_case=False)
# tokenizer = BertTokenizerFast(vocab_file=vocab_file, max_len=token_max_len, do_lower_case=False)


# speical 토큰 계수 + vocab 계수 - 이미 vocab에 포함된 speical 토큰 계수(5)
vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5 + 1
#vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5
print('special_token_size: {}, tokenizer.vocab_size: {}'.format(len(tokenizer.all_special_tokens), tokenizer.vocab_size))
print('vocab_size: {}'.format(vocab_size))
print('tokenizer_len: {}'.format(len(tokenizer)))


special_token_size: 27, tokenizer.vocab_size: 167537
vocab_size: 167560
tokenizer_len: 167550


In [6]:
def load_model():
    # 모델 로딩 further pre-training 
    #config = BertConfig.from_pretrained(model_path)
    #model = BertForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path), config=config) 
    model = BertForMaskedLM.from_pretrained(model_path)    

    #################################################################################
    # 모델 embedding 사이즈를 tokenizer 크기 만큼 재 설정함.
    # 재설정하지 않으면, 다음과 같은 에러 발생함
    # CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)` CUDA 에러가 발생함
    #  indexSelectLargeIndex: block: [306,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
    #
    #     해당 오류는 기존 Embedding(8002, 768, padding_idx=1) 처럼 입력 vocab 사이즈가 8002인데,
    #     0~8001 사이를 초과하는 word idx 값이 들어가면 에러 발생함.
    #################################################################################
    model.resize_token_embeddings(len(tokenizer))

    model.to(device)
    
    return model

In [7]:
def build_dataset(batch_size):
    from torch.utils.data import DataLoader, RandomSampler
    import sys
    sys.path.append('..')
    from myutils import MLMDataset
  
    # 각 스페셜 tokenid를 구함
    CLStokenid = tokenizer.convert_tokens_to_ids('[CLS]')
    SEPtokenid = tokenizer.convert_tokens_to_ids('[SEP]')
    UNKtokenid = tokenizer.convert_tokens_to_ids('[UNK]')
    PADtokenid = tokenizer.convert_tokens_to_ids('[PAD]')
    MASKtokenid = tokenizer.convert_tokens_to_ids('[MASK]')
    print('CLSid:{}, SEPid:{}, UNKid:{}, PADid:{}, MASKid:{}'.format(CLStokenid, SEPtokenid, UNKtokenid, PADtokenid, MASKtokenid))

    #===============================================================================
    # 학습 dataloader 생성
    train_dataset = MLMDataset(corpus_path = input_corpus,
                               tokenizer = tokenizer, 
                               CLStokeinid = CLStokenid ,   # [CLS] 토큰 id
                               SEPtokenid = SEPtokenid ,    # [SEP] 토큰 id
                               UNKtokenid = UNKtokenid ,    # [UNK] 토큰 id
                               PADtokenid = PADtokenid,    # [PAD] 토큰 id
                               Masktokenid = MASKtokenid,   # [MASK] 토큰 id
                               max_sequence_len=token_max_len,  # max_sequence_len)
                               mlm_probability=0.15,
                               overwrite_cache=False
                              )


    # 학습 dataloader 생성
    # => tenosor로 만듬
    train_loader = DataLoader(train_dataset, 
                              batch_size=batch_size, 
                              #shuffle=True, # dataset을 섞음
                              sampler=RandomSampler(train_dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                              num_workers=3
                             )
    #===============================================================================

    #===============================================================================
    # eval dataloader 생성
    eval_dataset = MLMDataset(corpus_path = eval_corpus,
                               tokenizer = tokenizer, 
                               CLStokeinid = CLStokenid ,   # [CLS] 토큰 id
                               SEPtokenid = SEPtokenid ,    # [SEP] 토큰 id
                               UNKtokenid = UNKtokenid ,    # [UNK] 토큰 id
                               PADtokenid = PADtokenid,    # [PAD] 토큰 id
                               Masktokenid = MASKtokenid,   # [MASK] 토큰 id
                               max_sequence_len=token_max_len,  # max_sequence_len)
                               mlm_probability=0.15,
                               overwrite_cache=False
                              )


    # eval dataloader 생성
    # => tenosor로 만듬
    eval_loader = DataLoader(eval_dataset, 
                              batch_size=batch_size, 
                              #shuffle=True, # dataset을 섞음
                              sampler=RandomSampler(eval_dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                              num_workers=3
                             )
     #===============================================================================

    #print(train_dataset[0])
    
    return train_loader, eval_loader

In [8]:
def Train_epoch(config,
                model,
                train_loader,
                eval_loader):
    
    ##################################################
    epochs = config.epochs            # epochs
    learning_rate = config.learning_rate  # 학습률
    #p_itr = 15000           # 손실률 보여줄 step 수
    #save_steps = 50000     # 50000 step마다 모델 저장
    ##################################################

    # optimizer 적용
    optimizer = AdamW(model.parameters(), 
                     lr=learning_rate, 
                     eps=1e-8) # 0으로 나누는 것을 방지하기 위한 epsilon 값(10^-6 ~ 10^-8 사이 이값 입력합)

    # 총 훈련과정에서 반복할 스탭
    total_steps = len(train_loader)*epochs
    warmup_steps = total_steps * 0.1 #10% of train data for warm-up
    
    # 손실률 보여줄 step 수
    p_itr = int(len(train_loader)*0.1)  
    
    # step마다 모델 저장
    save_steps = int(total_steps * 0.2)
    
    # 스캐줄러 생성
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps=warmup_steps, 
                                                num_training_steps=total_steps)

    itr = 1
    total_loss = 0
    total_len = 0
    total_correct = 0
    list_training_loss = []
    list_acc_loss = []
    list_validation_acc_loss = []

    model.zero_grad()# 그래디언트 초기화
    for epoch in tqdm(range(epochs)):

        # model.train() # 훈련모드로 변환
        for data in tqdm(train_loader):
            model.train() # 훈련모드로 변환
            #optimizer.zero_grad()
            model.zero_grad()# 그래디언트 초기화

            # 입력 값 설정
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            token_type_ids = data['token_type_ids'].to(device)       
            labels = data['labels'].to(device)
            #print('Labels:{}'.format(labels))

            # 모델 실행
            outputs = model(input_ids=input_ids, 
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            labels=labels)

            # 출력값 loss,logits를 outputs에서 얻어옴
            loss = outputs.loss
            logits = outputs.logits
            #print('Loss:{}, logits:{}'.format(loss, logits))

            # optimizer 과 scheduler 업데이트 시킴
            loss.backward()   # backward 구함
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   # 그래디언트 클리핑 (gradient vanishing이나 gradient exploding 방지하기 위한 기법)
            optimizer.step()  # 가중치 파라미터 업데이트(optimizer 이동)
            scheduler.step()  # 학습률 감소

            # ***further pretrain 에는 손실률 계산을 넣지 않음
            # 정확도 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.

            # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
            # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
            with torch.no_grad():
                
                # 손실(loss) 계산 
                total_loss += loss.item()
                
                # 정확도(Accurarcy) 계산
                pred = torch.argmax(logits, dim=2)
                correct = pred.eq(labels)
                total_correct += correct.sum().item()
                total_len += len(labels) * token_max_len   
    
                # 주기마다 test(validataion) 데이터로 평가하여 손실류 계산함.
                if itr % p_itr == 0:
                    logger.info('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Train Acc: {:.4f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))

                    list_training_loss.append(total_loss/p_itr)
                    
                    # wandb 로그 기록
                    wandb.log({"Loss": total_loss/p_itr,
                              "train_accuracy": total_correct/total_len
                              })
                        
                    total_loss = 0
                    total_len = 0
                    total_correct = 0
                    
                    ####################################################################
                    # 주기마다 eval(validataion) 데이터로 평가하여 손실류 계산함.
                    # 평가 시작
                    model.eval()

                    total_test_correct = 0
                    total_test_len = 0

                    #for data in tqdm(eval_loader):
                    for data in eval_loader:
                        # 입력 값 설정
                        input_ids = data['input_ids'].to(device)
                        attention_mask = data['attention_mask'].to(device)
                        token_type_ids = data['token_type_ids'].to(device)       
                        labels = data['labels'].to(device)

                        # 손실률 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
                        # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
                        # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
                        with torch.no_grad():
                            # 모델 실행
                            outputs = model(input_ids=input_ids, 
                                            attention_mask=attention_mask,
                                            token_type_ids=token_type_ids,
                                            labels=labels)

                            # 출력값 loss,logits를 outputs에서 얻어옴
                            #loss = outputs.loss
                            logits = outputs.logits

                            # 정확도(Accurarcy) 계산
                            pred = torch.argmax(logits, dim=2)
                            correct = pred.eq(labels)
                            total_test_correct += correct.sum().item()
                            total_test_len += len(labels) * token_max_len 

                    list_validation_acc_loss.append(total_test_correct/total_test_len)
                    logger.info("[Epoch {}/{}] Validatation Accuracy:{}".format(epoch+1, epochs, total_test_correct / total_test_len))

                    # wandb 로그 기록
                    wandb.log({"val_accuracy": total_test_correct / total_test_len})

                    ####################################################################

                '''
                if itr % save_steps == 0:
                    #전체모델 저장
                    TMP_OUT_PATH = OUTPATH + str(itr)
                    os.makedirs(TMP_OUT_PATH, exist_ok=True)
                    # save_pretrained 로 저장하면 config.json, pytorch_model.bin 2개의 파일이 생성됨
                    model.save_pretrained(TMP_OUT_PATH)
                    #torch.save(model, TMP_OUT_PATH + 'pytorch_model.bin') 

                    # tokeinizer 파일 저장(vocab)
                    VOCAB_PATH = TMP_OUT_PATH
                    #os.makedirs(VOCAB_PATH)
                    tokenizer.save_pretrained(VOCAB_PATH)

                    logger.info('Iteration {} -> save model:{}'.format(itr, TMP_OUT_PATH))
                '''
            itr+=1


In [9]:
# 훈련
def train(config=None):
    # wandb 초기화
    with wandb.init(config=config):
        config = wandb.config
        
        # 데이터 로더 생성
        train_loader, eval_loader = build_dataset(config.batch_size)
        
        # 모델 로딩
        tmodel = load_model()
        
        # 훈련 시작 
        Train_epoch(config, tmodel, train_loader, eval_loader)

In [12]:
# 훈련을 시작 (총 3번(epoch이 아님))
wandb.agent(sweep_id, train, count=5)

wandb: Agent Starting Run: hxjl1sxx with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	learning_rate: 3.880686219829353e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


2022-04-15 10:36:12,821 - bwpdataset - INFO - ==>[Start] cached file read: ../korpora/kowiki_20190620/cached_lm_BertTokenizer_128_wiki_20190620_small.txt


CLSid:101, SEPid:102, UNKid:100, PADid:0, MASKid:103
*corpus:../korpora/kowiki_20190620/wiki_20190620_small.txt
*max_sequence_len:128
*mlm_probability:0.15
*CLStokenid:101, SEPtokenid:102, UNKtokenid:100, PADtokeinid:0, Masktokeid:103


2022-04-15 10:36:13,071 - bwpdataset - INFO - <==[End] Loading features from cached file ../korpora/kowiki_20190620/cached_lm_BertTokenizer_128_wiki_20190620_small.txt [took 0.248 s]


*corpus:../korpora/kowiki_20190620/wiki_eval_test.txt
*max_sequence_len:128
*mlm_probability:0.15
*CLStokenid:101, SEPtokenid:102, UNKtokenid:100, PADtokeinid:0, Masktokeid:103
*total_line: 114


...

  0%|          | 0/307 [00:00<?, ?it/s]

2022-04-15 11:08:17,871 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Iteration 630 -> Train Loss: 0.0722, Train Acc: 0.9867
2022-04-15 11:08:18,652 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Validatation Accuracy:0.9772203947368421
2022-04-15 11:08:26,803 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Iteration 660 -> Train Loss: 0.0676, Train Acc: 0.9877
2022-04-15 11:08:27,577 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Validatation Accuracy:0.9773848684210527
2022-04-15 11:08:35,914 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Iteration 690 -> Train Loss: 0.0640, Train Acc: 0.9885
2022-04-15 11:08:36,740 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Validatation Accuracy:0.9771381578947368
2022-04-15 11:08:45,022 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Iteration 720 -> Train Loss: 0.0679, Train Acc: 0.9875
2022-04-15 11:08:45,791 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Validatation Accuracy:0.9770559210526316
2022-04-15 11:08:54,095 - wb-bert-fpt-mlm - INFO - [Epoch 3/4] Iteration 750 -> Train Loss: 0.0654, Train Acc: 0

  0%|          | 0/307 [00:00<?, ?it/s]

2022-04-15 11:09:48,892 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Iteration 930 -> Train Loss: 0.0633, Train Acc: 0.9882
2022-04-15 11:09:49,686 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Validatation Accuracy:0.9774671052631579
2022-04-15 11:09:57,990 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Iteration 960 -> Train Loss: 0.0567, Train Acc: 0.9893
2022-04-15 11:09:58,729 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Validatation Accuracy:0.9775493421052631
2022-04-15 11:10:07,011 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Iteration 990 -> Train Loss: 0.0521, Train Acc: 0.9904
2022-04-15 11:10:07,774 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Validatation Accuracy:0.9772203947368421
2022-04-15 11:10:16,028 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Iteration 1020 -> Train Loss: 0.0563, Train Acc: 0.9894
2022-04-15 11:10:16,781 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Validatation Accuracy:0.9774671052631579
2022-04-15 11:10:25,101 - wb-bert-fpt-mlm - INFO - [Epoch 4/4] Iteration 1050 -> Train Loss: 0.0565, Train Acc:

Loss,█▆▅▄▄▄▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▂▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████▇█
val_accuracy,▁▄▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇▇▇▇██████████████
Loss,0.05523
train_accuracy,0.98931
val_accuracy,0.97747


In [11]:
'''
### 전체모델 저장
os.makedirs(OUTPATH, exist_ok=True)
#torch.save(model, OUTPATH + 'pytorch_model.bin') 
# save_pretrained 로 저장하면 config.json, pytorch_model.bin 2개의 파일이 생성됨
model.save_pretrained(OUTPATH)

# tokeinizer 파일 저장(vocab)
VOCAB_PATH = OUTPATH
tokenizer.save_pretrained(VOCAB_PATH)
'''

"\n### 전체모델 저장\nos.makedirs(OUTPATH, exist_ok=True)\n#torch.save(model, OUTPATH + 'pytorch_model.bin') \n# save_pretrained 로 저장하면 config.json, pytorch_model.bin 2개의 파일이 생성됨\nmodel.save_pretrained(OUTPATH)\n\n# tokeinizer 파일 저장(vocab)\nVOCAB_PATH = OUTPATH\ntokenizer.save_pretrained(VOCAB_PATH)\n"